In [1]:
import ee
import geemap
import numpy as np
import pandas as pd

ee.Authenticate()
ee.Initialize()

dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

ModuleNotFoundError: No module named 'ee'

In [2]:
def land_cover(longitude, latitude):

    point = ee.Geometry.Point(longitude, latitude)

    # Filter the collection to the 2016 product.
    nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '2016')).first()

    # Select the land cover band.
    landcover = nlcd2016.select('landcover')

    # Sample the land cover data at the specified point.
    landcover_value = landcover.sample(point, scale=30).first().get('landcover').getInfo()
    
    return landcover_value

##### Try to get data for California on a grid with lat_delta and lon_delta resolution
lat_delta = 0.25
lon_delta = 0.25


#California box
top    = 42.009518
bottom = 32.534156
left   = -124.409591 
right  =  -114.131211

lats = list(np.arange(bottom, top, lat_delta))  # from southernmost to northernmost points with a 5-degree step
lons = list(np.arange(left, right, lon_delta))  # from westernmost to easternmost points with a 5-degree step

latitude = []
longitude = []
land_cover_data = []


for lat in lats:
    for lon in lons:
        try:
            land_cover_data.append(land_cover(lon, lat))
            latitude.append(lat)
            longitude.append(lon)
        except:
            land_cover_data.append(np.nan)
            latitude.append(lat)
            longitude.append(lon)

NameError: name 'np' is not defined

In [13]:
data = pd.DataFrame({'latitude':latitude, 'longitude': longitude, 'land_cover': land_cover_data})

In [14]:
data['land_cover'].notnull().sum()

1192

In [16]:
data[data['land_cover'].notnull()]

,latitude,longitude,land_cover
29,32.534156,-117.159591,11.0
39,32.534156,-114.659591,52.0
40,32.534156,-114.409591,71.0
41,32.534156,-114.159591,71.0
66,32.784156,-118.409591,11.0
...,...,...,...
1591,41.784156,-115.159591,52.0
1592,41.784156,-114.909591,52.0
1593,41.784156,-114.659591,52.0
1594,41.784156,-114.409591,71.0


In [21]:
data.to_csv('land_cover.csv', index = False)